In [1857]:
#priority queue with min heap, where min priority node at at the root node and max priority at one of the sterile/non-leaf nodes
#however this code is adjusted to account for using a priority queue with a tuple of the task id and the
#priority_score instead of just the priority score alone
class Priority_queue:
    import math
    def __init__(self):
        self.lst = []
    
    def display(self):
        return self.lst

    def swap(self, i, j):
        self.lst[i], self.lst[j] = self.lst[j], self.lst[i]

    def check_left(self, index):
        if 2*index + 1 <= len(self.lst) - 1:
            return True
        else:
            return False
    def check_right(self, index):
        if 2*index + 2 <= len(self.lst) - 1:
            return True
        else:
            return False

    #return the parent node for a given child at index
    def parent(self, index):
        if index != 0:
            return self.lst[math.floor((index - 1) /2)]

    #return left and right child nodes for a given parent at index
    def left(self, index):
        if self.check_left(index):
            return self.lst[2*index + 1]
    def right(self, index):
        if self.check_right(index):
            return self.lst[2*index + 2]

    #return the parent node index for a given child at index
    def parent_index(self, index):
        if index != 0:
            return math.floor((index - 1) /2)

    #return left and right child node indices for a given parent at index
    def left_index(self, index):
        if self.check_left(index):
            return 2*index + 1
    def right_index(self, index):
        if self.check_right(index):
            return 2*index + 2


    #returns indices of all the nodes without any left or right children  (non-leaf nodes)
    #this mehtod is EXTREMELY important in searching for the max priority node. this narrows down the search process
    #to include the steriles only
    def steriles(self):
        sterile_node_indices = []
        for node_index in range(len(self.lst)):
            if not self.check_left(node_index) and not self.check_right(node_index):
                sterile_node_indices.append(node_index)
        return sterile_node_indices

    #return the indices of all parent of any given child node in a heap, except for the root node
    def parents_indices(self, index):
        output_indices = []
        while self.parent_index(index) != None:
            output_indices.append(self.parent_index(index))
            index = output_indices[-1]
        return output_indices
    
    #this function bubbles up the max element at any tree branch until it reaches the top or its sorted place
    def bubble_min_up(self, child_index):
        local_child = self.lst[child_index]
        #print("passed child index:", child_index, ",lst:", self.lst, ",local child:", local_child)

        #storing all the parents indices for the given child
        parent_indices = self.parents_indices(child_index)
        
        #bubbling up the child to the top
        #print("child:", local_child, "parents:" ,parent_indices)
        for parent_index in parent_indices:
            parent = self.lst[parent_index]
            #print("parent_index:", parent_index)
            if local_child[1] <= parent[1]:
                self.swap(child_index, parent_index)
                child_index = parent_index
                #print(local_child, "at index", child_index, "AND", parent, "at index", parent_index, "swapping!")
                #print("lst after swapping:", self.lst)
                #print("#############################")
        #print("lst after bubbling the child", self.lst)

    #this method ensures that ALL the parents (at the upper levels) of a given node at index maintain the min heap property
    def min_heapify(self, index):
        if index < 0:
            print("Only positive indices are allowed!", "Index rejected:", index)
            return "-ve index error"
        parent_indices = self.parents_indices(index)
        #child = self.lst[index]
        branch = [index] + parent_indices
        #sorting the branch ascendingly (max at bottom) by bubbling up each child in the branch, starting from the bottom
        for node_index in branch:
            #print("current_child_index:", node_index)
            self.bubble_min_up(node_index)

    #this function adopts a node at the end of the heap. notice that it can't add the element at an index because
    # there isn't any use for that with priority queues
    def adopt(self, adoptee):
        if adoptee[1] < 0:
            print("Only positive adoptee nodes are allowed!", "Adoptee node rejected:", adoptee)
            return "-ve node error"
        self.lst.append(adoptee)
        self.min_heapify(len(self.lst) - 1)
    
    def lst_to_heap(self, lst):
        for node in lst:
            self.adopt(node)
        
    #add heapify
    #this method delets a node at a given index and heapifies all the sterile/non-leaf nodes afterwards
    #the method deletes nodes through swapping them with the last element in the heap and then heapifying the whole tree again
    #notice that I heapify the whole tree because for now, the program is agnostic towards the descendents of each node (I tried to write a
    # method but failed) have we known the descendents, we'd have cut the runtime by half as we'd only heapify the sterile descendents of the
    #node at the deleted node index, instead of heapifying the whole tree
    def delete(self, index):
        deletee = self.lst[index]
        self.swap(index, len(self.lst) - 1)
        del self.lst[-1]

        #heapifying the tree after deleting
        steriles = self.steriles()
        for sterile_node in steriles:
            self.min_heapify(sterile_node)

    #deletes a node at index and and returns its value (pop) and then heapify the tree
    def pop(self, index):
        poppee = self.lst[index]
        self.delete(index)
        return poppee
    
    #return the index of the max priority node (between the sterile/non-leaf nodes in the heapified lst)
    def max_priority_index(self):
        #reversing the list to ensure returning the most remote max element/sterile max node to ease the heapifying process later by      
        steriles = list(reversed(self.steriles().copy()))
        max = -1
        max_index = "NA"
        for sterile_node_index in steriles:
            if self.lst[sterile_node_index][1] >= max:
                max_index = sterile_node_index
                max = self.lst[sterile_node_index][1]
                #print("current_max_index:", sterile_node_index, "max_value:", max)
        return max_index

    #return the index of the min priority node: root node
    def min_priority_index(self):
        return 0

    #return the index of max priority node
    def max_priority(self):
        return self.lst[self.max_priority_index()]
    #return the index of min priority node
    def min_priority(self):
        return self.lst[self.min_priority_index()]

    #pop the max priority node in the whole heap and then apply heapify (applied in pop)
    def pop_max(self):
        if len(self.lst) == 0:
            print("The queue is empty!")
            return "Empty queue error"
        max_index = self.max_priority_index()
        return self.pop(max_index)

    #pop the min priority node in the whole heap and then apply heapify (applied in pop)
    def pop_min(self):
        if len(self.lst) == 0:
            print("The queue is empty!")
            return "Empty queue error"
        min_index = self.min_priority_index()
        return self.pop(min_index)

    #this method returns the queue sorted in ascending order according to their priorities
    #this method can only be called after the lst_to_heap method is called
    def sorted_queue(self):
        import sys
        if len(self.display()) == 0:
            print("The queue is empty!")
            return "Empty queue error"
            sys.exit()
        sorted_nodes = []
        while len(self.display()) > 0:
            root_node = self.pop_min()
            #print(root_node)
            sorted_nodes.append(root_node)
        return sorted_nodes

In [663]:
#importing essential libraries
import pandas as pd
import matplotlib.pyplot as plt

In [1603]:
#Helper functions for Q1

#Here, I am using a nested list transposing function I orignally wrote for minerva's post summer assesment in 2020 and cited it in other CS
#assignemnt classes previously as CS50 Logical thinkin assignment(Badra, 2020). Notice that I am citing myself here as if I'd rewrote the
#this function from sratch it would have likely, unintentionally, looked the same as the one I wrote before anyways (#selfawareness).

#Transposing rows to columns (in this case) and can be used for columns to rows as well
def RowsToColumns(ColumnsLst):
    cell = 0
    RowsLst = []
    for i in range(len(ColumnsLst[0])):
        temp_row = []
        for column in ColumnsLst:
            temp_row.append(column[cell])
        cell += 1
        RowsLst.append(temp_row)
    return RowsLst

In [1988]:
#lst tasks format -> ["id" (int), "parent_task_description" (str), "duration" (float),"subtasks" (lst of str), "possible_multitasks" (lst of int), "depends_on" (lst of int), "overlaps" (lst of int)
# "multi_tasking" (bool), "status"(str), "due_time"(str), "importance"(int), "work"(bool), "priority_score"(float)]

tasks_lst = [[0, "description of parent task 0", 9, ["childtask1", "childtask2", "childtask3", "childtask4"], [], [2], [], False, 'not_yet_started', "15:00 25/10/21", 10, False, 0],
         [1, "description of parent task 1", 0.25, ["childtask1", "childtask2", "childtask3"], [], [3, 4], [], False, 'not_yet_started', "15:00 26/10/21", 10, True, 0],
         [2, "description of parent task 2", 5, ["childtask1", "childtask2", "childtask3"], [], [1, 2], [], False, 'not_yet_started', "15:00 27/10/21", 7, False, 0],
         [3, "description of parent task 3", 3, ["childtask1", "childtask2", "childtask3"], [], [1, 3], [], True, 'not_yet_started', "15:00 28/10/21", 5, True, 0],
         [4, "description of parent task 4", 2.5, ["childtask1", "childtask2", "childtask3"], [], [1], [], True, 'not_yet_started', "15:00 29/10/21", 6, False, 0],
         [5, "description of parent task 5", 0.5, ["childtask1", "childtask2", "childtask3"], [], [0], [], True, 'not_yet_started', "15:00 30/10/21", 2, False, 0],
]
#transposing the tasks lst in order to assign each column to a key in the tasks dict as values
tasks_lst_transposed = RowsToColumns(tasks_lst)
#transforming the tasks lst to a dict for ease of access later in the program
tasks_keys = ["id", "parent_task_description", "duration", "subtasks", "possible_multitasks", "depends_on", "overlaps","multi_tasking", "status", "due_time", "importance", "work", "priority_score"]

#converting the nested lst of tasks to a dict 
tasks_untouched = {tasks_keys[key_index]:tasks_lst_transposed[key_index] for key_index in range(len(tasks_keys))}

#dict to df for table display
tasks_df = pd.DataFrame(tasks_untouched)
tasks_df

,id,parent_task_description,duration,subtasks,possible_multitasks,depends_on,overlaps,multi_tasking,status,due_time,importance,work,priority_score
0,0,description of parent task 0,9.00,"[childtask1, childtask2, childtask3, childtask4]",[],[2],[],False,not_yet_started,15:00 25/10/21,10,False,0
1,1,description of parent task 1,0.25,"[childtask1, childtask2, childtask3]",[],"[3, 4]",[],False,not_yet_started,15:00 26/10/21,10,True,0
2,2,description of parent task 2,5.00,"[childtask1, childtask2, childtask3]",[],"[1, 2]",[],False,not_yet_started,15:00 27/10/21,7,False,0
3,3,description of parent task 3,3.00,"[childtask1, childtask2, childtask3]",[],"[1, 3]",[],True,not_yet_started,15:00 28/10/21,5,True,0
4,4,description of parent task 4,2.50,"[childtask1, childtask2, childtask3]",[],[1],[],True,not_yet_started,15:00 29/10/21,6,False,0
5,5,description of parent task 5,0.50,"[childtask1, childtask2, childtask3]",[],[0],[],True,not_yet_started,15:00 30/10/21,2,False,0


In [1418]:
#I realized that the time tracking operations as calculating elapsed time, time difference, elapsed days and printing time
#took me more than 100 lines so I decided to separate it in an independent class and inherit it in Scheduler
class Time_ops:
    def __init__(self):
        pass

    #"HH:MM DD/MM/YY"
    #I am adapting this function from a code I wrote for Variables with LBA assignmnt last year for CS50 (Badra, 2020)
    #this function returns the number of days for any month given as int in the format of MM
    def month_to_days(self, month):
        small_months = [4, 6, 9, 11]
        big_months = [1, 3, 5, 7, 8, 10, 12]
        months_lst = small_months + big_months + [2]
        months = {}
        for key in months_lst:
            if key in small_months:
                months[key] = 30
            elif key in big_months:
                months[key] = 31
            else:
                months[key] = 28.24255
        
        return months[month]

    #this funtion helps me calculate the time difference between any given dates through finding the number of days
    #in any given month
    #this function caclulates the days that passed to get to the current input month
    def elapsed_months_to_days(self, month_str):
        month_int = int(month_str)

        days_passed = 0
        for month in range(1, month_int + 1):
            #print(month, months[month])
            days_passed += self.month_to_days(month)
        return days_passed

    #this method calculates the amount of time between now and any given task at id
    def time_difference(self, current_time, task_id):
        t1 = current_time
        t2 = self.tasks["due_time"][task_id]
        #print(t1, t2)

        hours_diff = 0

        hours1_str = t1.split()[0].split(":")
        hours1 = int(hours1_str[0]) + int(hours1_str[1])/60
        #print("hours1:", hours1)

        dates1_str = t1.split()[1].split("/")
        days1 = int(dates1_str[0]) + self.elapsed_months_to_days(dates1_str[1]) + int(dates1_str[2])*365
        hours1 += 24*days1


        hours2_str = t2.split()[0].split(":")
        hours2 = int(hours2_str[0]) + int(hours2_str[1])/60
        #print("hours2:", hours2)

        dates2_str = t2.split()[1].split("/")
        days2 = int(dates2_str[0]) + self.elapsed_months_to_days(dates2_str[1]) + int(dates2_str[2])*365
        hours2 += 24*days2
        
        return round(hours2 - hours1, 2)
    
    #this function adds a given duration, in minutes, to any time given by "HH:MM DD/MM/YY"
    #notice that this function doesn't assume that tasks have to begin and end on the same day, instead they can take as long as they need
    #and the date will adjust accordingly
    def time_addition(self, date_time, duration):
        current_hours = int(date_time.split()[0].split(":")[0])
        current_minutes = int(date_time.split()[0].split(":")[1])
        current_day = int(date_time.split()[1].split("/")[0])
        current_month = int(date_time.split()[1].split("/")[1])
        current_year = int(date_time.split()[1].split("/")[2])

        extra_hours = duration // 60
        extra_minutes = duration - extra_hours*60
        extra_days = 0
        extra_months = 0
        extra_years = 0

        current_minutes += extra_minutes
        if current_minutes >= 60:
            current_minutes -= 60
            extra_hours +=1
        
        current_hours += extra_hours
        if current_hours >= 24:
            current_hours -= 24
            extra_days +=1

        days_in_current_month = self.month_to_days(current_month)
        current_day += extra_days
        if current_day >= days_in_current_month:
            current_day -= days_in_current_month
            extra_months +=1 
        
        current_month += extra_months
        if current_month >= 12:
            current_month -= 12
            extra_years +=1

        #this method assumes that we're limited to the 21st century
        current_year += extra_years

        return str(current_hours)+":"+str(current_minutes) + " " + str(current_day)+"/"+str(current_month)+"/"+str(current_year)

    #this method returns the time in the format: "HHhMMm DD/MM/YY"
    def print_time(self, time):
        hours_str = time.split()[0].split(":")
        hours, minutes = hours_str[0], hours_str[1]
        date = time.split()[1]

        return hours + "h" + minutes + "m" + " " + time.split()[1]

In [1944]:
#this class provides methods to updates the overlaps, possible_multitasks, and priority_score
#Run the updaters (decide_overlap, decide_multitasks, and decide_priority) before running any method in Scheduler
class Backend_update(Time_ops):
    def __init__(self, tasks_input, init_time, work_priority):
        super().__init__()
        self.tasks = tasks_input
        self.n_tasks = len(self.tasks["id"])
        self.time_now = init_time
        if work_priority:
            self.work_priority = True
        else:
            self.work_priority = False


    #this method returns the collective duartion of a group of tasks given by a list of their indices
    #this method will be used to decide whether to multitask or not in the decide_multitask method
    def tasks_duration(self, tasks_indices_lst):
        collective_duration = 0
        for task_index in tasks_indices_lst:
            collective_duration += tasks_untouched["duration"][task_index]
        
        return collective_duration
    
    #this method fills in the overlaps values for each task with the ids of overlapping tasks, determined with the due_time
    def decide_overlap(self):
        for task_index in range(self.n_tasks):
            #print(self.tasks["overlaps"])
            overlapping_tasks = []
            task_time = self.tasks["due_time"][task_index]
            #for each given task, append all the overlapping tasks
            #determine if the task is overlapping if the time difference between the current task and the other task candidate is bigger
            #than or equal to zero, if the opposite, the difference is less than 0, then the cadndidate task comes before the current task
            #and they're overlapping relation is flipped
            for task_cand_id in range(task_index + 1, self.n_tasks):
                if self.time_difference(task_time, task_cand_id) >= 0:
                    overlapping_tasks.append(task_cand_id)
                    
            self.tasks["overlaps"][task_index] = overlapping_tasks

    #this method updates the possible_multitask value for each task with indices of multitaskable tasks with each given task
    #to decide which tasks are multitaskable, it decides based upon: work, multi_tasking, overlaps, and duration       
    def decide_multitask(self):
        #go through each task, checking overlaps, other task's multi_task and work, current task mulit_task and duration! 
        #now what
        for task_index in range(self.n_tasks):
            #store the parent candidate duration for comparison later with the collective duration of candidate tasks for multitasking
            parent_task_duration = self.tasks["duration"][task_index]

            #print(self.tasks["possible_multitasks"])
            multi_tasks = []
            for overlap_task_index in self.tasks["overlaps"][task_index]:
                cand_task_duration = self.tasks["duration"][overlap_task_index]
                #only allow multitasking for overlapping tasks that are mutlitaskable and aren't work (as indicated by the user)
                if (self.tasks["work"][overlap_task_index] == False) and (self.tasks["multi_tasking"][overlap_task_index] == True):
                    #also check that the current task allows overlap
                    #notice that the task due earlier gets the other overlapping and multi_tasking-qualifying task appended to it
                    if self.tasks["multi_tasking"][task_index] == True:
                        #I wrote a seperate if statement here instead of an and for clarity and to clearly explain the key logic here
                        #ONLY append the task candidate for multitasking if adding would still mean that the parent task ends last or at
                        #the same time as the last multitasking task
                        #in other words, the collective duration of as multitasking tasks for any given ask HAS to be less than or equal
                        #the duration of the parent task
                        if self.tasks_duration(multi_tasks) + cand_task_duration <= parent_task_duration:
                            multi_tasks.append(overlap_task_index)
                    
            self.tasks["possible_multitasks"][task_index] = multi_tasks

    #depends_on sub priority score calculation
    #this method assigns the highest priority value (10) for values that are most mentioned in depends_on
    def depends_on_score_update(self):
        all_depend_ons = self.tasks["depends_on"]
        dependees = []
        for dependee in all_depend_ons:
            dependees += dependee

        #populating the dependees into a dict to count them
        dependee_dict = {}
        for task_ref in dependees:
            if task_ref in list(dependee_dict.keys()):
                dependee_dict[task_ref] += 1
            else:
                dependee_dict[task_ref] = 1

        #check if some task have no other dependent tasks and assign 0 to their value
        for index in self.tasks["id"]:
            if index not in list(dependee_dict.keys()):
                dependee_dict[index] = 0

        #finding the most and least repeated dependecy for dependency score mapping next
        dependencies_freq = list(dependee_dict.values())
        max_dependencies = max(dependencies_freq)
        min_dependencies = min(dependencies_freq)

        for index in self.tasks["id"]:
            dependees = dependee_dict[index]
            #this direct linear mapping method works in a similar way to the due_time one above
            #eqn: depends_on_score = (10/max_dependencies-min_dependencies)*dependees + (10*min_dependencies/(min_dependencies-max_dependencies))
            #the mapping process and graphs are included in a cell below
            depends_on_score = round(dependees*(10/(max_dependencies-min_dependencies)) + (10*min_dependencies/(min_dependencies-max_dependencies)), 2)
            
            #adding the depends on priority score to the overall priority score (1-50)
            self.tasks["priority_score"][index] += depends_on_score
            #print("dependees:", dependees, "depends_on_score:", depends_on_score)
    
    #this method calculates the priority score (1-60) of each task based on work, importance, due_time, duration, and possible_multitasks 
    def decide_priority(self):        
        for task_index in range(self.n_tasks):
            composite_task_priority = 0

            #work sub priority score calculation
            #decide whether to prioritize work or not depending on the work_priority bool input
            #if prioritizing work, then assign a sub priority value of 10 for work tasks, and 0 for the rest.
            #if not prioritizing work, then vice verse
            if self.tasks["work"][task_index] == True and self.work_priority:
                composite_task_priority += 10
                #print(self.tasks["work"][task_index], composite_task_priority)
            #############################################################################

            #importance sub priority score calculation
            #print("task importance:", self.tasks["importance"][task_index], ",current_composite:", composite_task_priority)
            composite_task_priority += self.tasks["importance"][task_index]
            #print("importance lst:", self.tasks["importance"], ",current_composite:", composite_task_priority)
            #############################################################################

            #due_time sub priority score calculation
            #calculating the time difference between now and each given task and finding the furthest and closest tasks
            task_time_left = self.time_difference(self.time_now, task_index)
            tasks_time_left = [self.time_difference(self.time_now, candidate_index) for candidate_index in range(self.n_tasks)]
            #print(tasks_time_left)
            max_time, min_time = max(tasks_time_left), min(tasks_time_left)

            #this line maps the closest/past due task to a due_time_score of 10 (max) and the furthest task to 0 (min)
            #through this equation: due_time_score = time_till_task*(10/min_time-max_time) + 10*max_time/max_time-min_time
            #the graph for this equation is included in a sepraten cell
            due_time_score = round(((10/(min_time-max_time))*task_time_left + (10*max_time/(max_time-min_time))), 2)
            composite_task_priority += due_time_score
            #print(task_index, due_time_score, composite_task_priority)
            #############################################################################

            #duration sub priority score calculation
            #this method follows the same priority calculation method using a linear mapping as used with the due_time
            task_duration = self.tasks["duration"][task_index]
            #print(self.tasks["duration"])
            max_duration, min_duration = max(self.tasks["duration"]), min(self.tasks["duration"])

            duration_score = round(((10/(min_duration-max_duration))*task_duration + (10*max_duration/(max_duration-min_duration))), 2)
            composite_task_priority += duration_score
            #print(task_index, duration_score, composite_task_priority)
            #############################################################################
            
            #possible_multitasks sub priority score calculation
            #give a higher priority for task with the largest number of possible_multitasks
            #print("possible_multitasks:", self.tasks["possible_multitasks"][task_index], ",current_composite:", composite_task_priority)
            task_n_possible_multitasks = len(self.tasks["possible_multitasks"][task_index])
            n_all_possible_multitasks = [len(self.tasks["possible_multitasks"][index]) for index in range(self.n_tasks)]
            
            max_possible_multitasks, min_possible_multitasks = max(n_all_possible_multitasks), min(n_all_possible_multitasks)
            #print(max_possible_multitasks, min_possible_multitasks)

            possible_multitasks_score = round(task_n_possible_multitasks*(10/(max_possible_multitasks-min_possible_multitasks)) + (10*min_possible_multitasks/(min_possible_multitasks-max_possible_multitasks)), 2)

            #print(possible_multitasks_score)
            composite_task_priority += possible_multitasks_score
            #############################################################################
            
            self.tasks["priority_score"][task_index] = composite_task_priority

        self.depends_on_score_update()

In [1989]:
class Scheduler(Priority_queue, Backend_update):
    import sys
    def __init__(self, tasks_input, init_time, work_priority):
        super().__init__()
        self.tasks = dict(tasks_input.copy())
        self.time_now = init_time

        self.n_tasks = len(self.tasks["id"])
        self.ordered_tuple_tasks = []
        self.ordered_indices = []
        if work_priority:
            self.work_priority = True
        else:
            self.work_priority = False
            
    #update the tasks dict overlaps, possible_multitasks, and priority score attributes for each tasks
    def decide_update(self):
        self.decide_overlap()
        self.decide_multitask()
        self.decide_priority()

    #this method updates the status for any given task by passing in status to the main self.tasks list
    def status_updater(self, index, status):
        if status not in ["not_yet_started", "in_progress", "completed"]:
            print("Please limit the input to one of these satuses:", ["not_yet_started", "in_progress", "completed"])
            return "status error"
        self.tasks["status"][index] = status
        
    #this function queues the tasks ONLY after the priority_score is computed
    #this method queues only the tasks with status 'not_yet_started'
    def queue(self):
        #populate the index_priority_tuples with the task id and its priority score tuples in this format: (id, priority_score)
        index_priority_tuples = [(self.tasks["id"][index], self.tasks["priority_score"][index]) for index in range(len(self.tasks["id"])) if self.tasks["status"][index] == 'not_yet_started']
        #print(index_priority_tuples)
        self.lst_to_heap(index_priority_tuples)
    

    #this method initializes the planner by updating the tasks priority score and pushing tasks into queue
    #then, sorted_queue transforms the queue into a list sorted in ascending order according to the priority
    def initialize(self):
        empty = True
        for task_status in self.tasks["status"]:
            if task_status != "completed":
                empty = False
                break
        if empty:
            sys.exit("All the tasks were previously completed! Please update and rerun the tasks list above")
        
        #update all the backend attributes before executing the planner
        self.decide_update()
        #queue according to the priority_score calculated by the decide_update method
        self.queue()
        self.ordered_tuple_tasks = self.sorted_queue()        

        #orders the list of multitasks for all tasks according to the priority queue
        self.order_multitasks()

    #this method can only be run after initialize()
    #this method sorts the list of multitasking tasks for each task by priority_score, where the highest priority is at the ends
    #and the lowest priority is at the beginning

    #run this method in plan() directly after initialize()
    #populate the self.ordered_indices with the indices sorted ascendingly by the priority, where the highest priority is at the end
    def order_multitasks(self):
        #go over the each task, for each multitaskie, go over the ordered_tasks, when you find the multitaskie in the ordered_tasks, then put
        #append it into the ordered multitask lst
        #then update the list of multitas_possibles
        self.ordered_indices =  [self.ordered_tuple_tasks[index][0] for index in range(len(self.ordered_tuple_tasks))]
        #print(self.ordered_indices)
        #print(self.ordered_tuple_tasks, self.ordered_indices)
        for task_index in self.ordered_indices:
            sorted_multitasks = []
            unsorted_multitasks = self.tasks["possible_multitasks"][task_index]

            #print(self.tasks["possible_multitasks"][task_index])
            #making use of the already sorted (according to priority) queue to sort the multitaskies according to priority as well
            #at the end, the list of multitask will be sorted according to the priority queue instead of creating its own queue to sort
            #them according to priority
            #the highest priority multitaskie will be at the end
            for sorted_task in self.ordered_indices:
                if sorted_task in unsorted_multitasks:
                    sorted_multitasks.append(sorted_task)
            
            #print(sorted_multitasks)
            self.tasks["possible_multitasks"][task_index] = sorted_multitasks

    def execute_multitasks(self, lst_multitasks_indices):
        n_multitasks = len(lst_multitasks_indices)
        for multitask in range(n_multitasks):
            current_multitask_index = lst_multitasks_indices.pop()

            self.status_updater(current_multitask_index, "in_progress")
            current_multitask_duration = int(self.tasks["duration"][current_multitask_index])
            multitask_desc = self.tasks["parent_task_description"][current_multitask_index]
            
            print("🔀 Simple Scheduler at time", self.print_time(self.time_now), "started multitasking with task:", multitask_desc)

            self.ordered_indices.remove(current_multitask_index)
            self.status_updater(current_multitask_index, "completed")
            self.time_now = self.time_addition(self.time_now, current_multitask_duration)

    #this function is where all the methods come together
    #run the initialize inside here before andything!
    def plan(self):
        #initializing the priority queue
        self.initialize()

        ordered_indices_copy = self.ordered_indices.copy()

        print("📜 Statuses (id, status):", list(zip(ordered_indices_copy, self.tasks["status"])))
        
        #dequeuing highest priority tasks from the queue until it's empty
        while len(self.ordered_indices) > 0:
            #print(self.ordered_indices)
            #add the duration priority logic here
            #storing the main attributes for each task
            current_task_index =  self.ordered_indices.pop()
            current_task_duration = int(self.tasks["duration"][current_task_index])

            #updating the task status
            self.status_updater(current_task_index, "in_progress")

            print("📜 Statuses (id, status):", list(zip(ordered_indices_copy, self.tasks["status"])))
            #storing the description for the tasks and subtasks
            main_task_desc = self.tasks["parent_task_description"][current_task_index]
            current_tasks_desc = [main_task_desc]
            current_subtasks_desc = self.tasks["subtasks"][current_task_index]
            current_tasks_desc += current_subtasks_desc
            #print(self.tasks["subtasks"][current_task_index])
            
            #the combined list of tasks and subtasks (if there are any)
            current_tasks_desc + self.tasks["subtasks"][current_task_index]
            
            #this loop iterates over all the description of the elements in current_tasks_desc and prints the start of executio messages
            for task_index in range(len(current_tasks_desc)):
                if len(current_tasks_desc) == 1:
                    print("⏱ Simple Scheduler at time", self.print_time(self.time_now), "started executing task:", current_tasks_desc[task_index])

                elif task_index == 0:
                    print("⏱ Simple Scheduler at time", self.print_time(self.time_now), "started executing parent task:", current_tasks_desc[task_index])
                    
                    #executing multitasks directly after executing the parent tasks
                    self.execute_multitasks(self.tasks["possible_multitasks"][current_task_index])
                else:
                    print("⏱ Simple Scheduler started executing subtask:", current_tasks_desc[task_index])
            
            #updating the task status
            self.status_updater(current_task_index, "completed")
            print("📜 Statuses (id, status):", list(zip(ordered_indices_copy, self.tasks["status"])))
            
            #updating the time and indicating the finished task
            self.time_now = self.time_addition(self.time_now, current_task_duration)

            if len(current_tasks_desc) == 1:
                #print the main task
                print("✅ Completed Task:", current_tasks_desc[0] ,"at", self.print_time(self.time_now))
            else:
                #print with subtasks
                print("✅ Completed Task:", current_tasks_desc[0], "and subtasks:", current_subtasks_desc ,"at", self.print_time(self.time_now))
            print()
            
        print("⌛🌟 You are done! No more items are left in the queue")       

In [1990]:
s = Scheduler(tasks_untouched.copy(), "03:20 25/10/21", True)
#s.time_difference("00:00 24/10/21", 1)

#print(s.display())

#s.plan()
#s.time_addition("23:20 22/10/21", 150)

#s.print_time("23:22 22/10/21")

#[not_yet_started, in_progress, completed]
#s.status_updater(2, "not_yet_started")


#print(s.tasks["status"])


#print(s.tasks["priority_score"])
#s.decide_priority()
#print(s.tasks["priority_score"])

s.plan()

📜 Statuses (id, status): [(5, 'not_yet_started'), (0, 'not_yet_started'), (4, 'not_yet_started'), (2, 'not_yet_started'), (3, 'not_yet_started'), (1, 'not_yet_started')]
📜 Statuses (id, status): [(5, 'not_yet_started'), (0, 'in_progress'), (4, 'not_yet_started'), (2, 'not_yet_started'), (3, 'not_yet_started'), (1, 'not_yet_started')]
⏱ Simple Scheduler at time 03h20m 25/10/21 started executing parent task: description of parent task 1
⏱ Simple Scheduler started executing subtask: childtask1
⏱ Simple Scheduler started executing subtask: childtask2
⏱ Simple Scheduler started executing subtask: childtask3
📜 Statuses (id, status): [(5, 'not_yet_started'), (0, 'completed'), (4, 'not_yet_started'), (2, 'not_yet_started'), (3, 'not_yet_started'), (1, 'not_yet_started')]
✅ Completed Task: description of parent task 1 and subtasks: ['childtask1', 'childtask2', 'childtask3'] at 3h20m 25/10/21

📜 Statuses (id, status): [(5, 'not_yet_started'), (0, 'completed'), (4, 'not_yet_started'), (2, 'in_pro

In [1991]:
x = pd.DataFrame(s.tasks)
x

,id,parent_task_description,duration,subtasks,possible_multitasks,depends_on,overlaps,multi_tasking,status,due_time,importance,work,priority_score
0,0,description of parent task 0,9.00,"[childtask1, childtask2, childtask3, childtask4]",[],[2],"[1, 2, 3, 4, 5]",False,completed,15:00 25/10/21,10,False,23.33
1,1,description of parent task 1,0.25,"[childtask1, childtask2, childtask3]",[],"[3, 4]","[2, 3, 4, 5]",False,completed,15:00 26/10/21,10,True,48.00
2,2,description of parent task 2,5.00,"[childtask1, childtask2, childtask3]",[],"[1, 2]","[3, 4, 5]",False,completed,15:00 27/10/21,7,False,24.24
3,3,description of parent task 3,3.00,"[childtask1, childtask2, childtask3]",[],"[1, 3]","[4, 5]",True,completed,15:00 28/10/21,5,True,42.53
4,4,description of parent task 4,2.50,"[childtask1, childtask2, childtask3]",[5],[1],[5],True,completed,15:00 29/10/21,6,False,23.76
5,5,description of parent task 5,0.50,"[childtask1, childtask2, childtask3]",[],[0],[],True,completed,15:00 30/10/21,2,False,11.71
